# Capstone Project 2 - DataSpark

In [1]:
pip install chardet

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: C:\Users\DELL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# Detect file encoding

import chardet

def detect_encoding(file):
    with open(file, 'rb') as file:
        result = chardet.detect(file.read())
        encoding = result['encoding']
        print(f"Detected encoding for {file} : {encoding}")

list_of_csvs=[r'uncleaned_datasets/Customers.csv',r'uncleaned_datasets/Exchange_rates.csv',r'uncleaned_datasets/Products.csv',r'uncleaned_datasets/Sales.csv',r'uncleaned_datasets/Stores.csv']

for i in list_of_csvs:
    detect_encoding(i)

Detected encoding for <_io.BufferedReader name='uncleaned_datasets/Customers.csv'> : Windows-1252
Detected encoding for <_io.BufferedReader name='uncleaned_datasets/Exchange_rates.csv'> : ascii
Detected encoding for <_io.BufferedReader name='uncleaned_datasets/Products.csv'> : ascii
Detected encoding for <_io.BufferedReader name='uncleaned_datasets/Sales.csv'> : ascii
Detected encoding for <_io.BufferedReader name='uncleaned_datasets/Stores.csv'> : utf-8


## Data Cleaning and Preparation

### 1] Customers Analysis

In [3]:
import pandas as pd

df_customers=pd.read_csv(list_of_csvs[0], encoding='latin-1')
df_customers


,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,7/3/1939
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,9/27/1979
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,5/26/1947
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,9/17/1957
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,11/19/1965
...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Denisa Duková,Houston,TX,Texas,77017,United States,North America,3/25/1936
15262,2099618,Male,Justin Solórzano,Mclean,VA,Virginia,22101,United States,North America,2/16/1992
15263,2099758,Male,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,11/9/1937
15264,2099862,Female,Lorenza Rush,Riverside,CA,California,92501,United States,North America,10/12/1937


In [4]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15266 entries, 0 to 15265
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CustomerKey  15266 non-null  int64 
 1   Gender       15266 non-null  object
 2   Name         15266 non-null  object
 3   City         15266 non-null  object
 4   State Code   15256 non-null  object
 5   State        15266 non-null  object
 6   Zip Code     15266 non-null  object
 7   Country      15266 non-null  object
 8   Continent    15266 non-null  object
 9   Birthday     15266 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.2+ MB


In [5]:
# Convert Birthday to Datetime format

df_customers['Birthday'] = pd.to_datetime(df_customers['Birthday'], format='%m/%d/%Y')

In [6]:
# Check for null values

df_customers.isna().sum()

CustomerKey     0
Gender          0
Name            0
City            0
State Code     10
State           0
Zip Code        0
Country         0
Continent       0
Birthday        0
dtype: int64

In [7]:
# Null values from customer table

df_customers[df_customers.isnull().any(axis=1)]

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
5304,729681,Female,Rossana Padovesi,Polvica,NaN,Napoli,80035,Italy,Europe,1981-04-18
5316,732289,Male,Indro Piccio,Varcaturo,NaN,Napoli,80014,Italy,Europe,1949-02-24
5372,742042,Male,Amaranto Loggia,Casaferro,NaN,Napoli,80034,Italy,Europe,1936-03-14
5377,742886,Female,Edmonda Capon,Terzigno,NaN,Napoli,80040,Italy,Europe,1963-08-06
5378,743343,Female,Ambra Sagese,Pomigliano D'Arco,NaN,Napoli,80038,Italy,Europe,1961-01-05
5485,759705,Male,Callisto Lo Duca,Casilli,NaN,Napoli,80047,Italy,Europe,1976-08-28
5525,765589,Male,Michelino Lucchesi,Pompei Scavi,NaN,Napoli,80045,Italy,Europe,1947-11-13
5531,766410,Male,Adelmio Beneventi,Licola,NaN,Napoli,80078,Italy,Europe,1940-01-13
5631,781667,Female,Ilda Manna,Napoli,NaN,Napoli,80134,Italy,Europe,1977-05-08
5695,789177,Male,Calogero Folliero,Mariglianella,NaN,Napoli,80030,Italy,Europe,2000-03-03


In [8]:
# Replacing State Code null values for Naploli as NA as it is the state code for Napoli 
df_customers.fillna('NA', inplace=True)

# Removing spaces in column names as it is better for sql
df_customers.columns = df_customers.columns.str.replace(' ', '')

In [9]:
# Checking for duplicated values

df_customers.duplicated().sum()

0

In [10]:
df_customers

,CustomerKey,Gender,Name,City,StateCode,State,ZipCode,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,1939-07-03
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,1979-09-27
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,1947-05-26
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,1957-09-17
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,1965-11-19
...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Denisa Duková,Houston,TX,Texas,77017,United States,North America,1936-03-25
15262,2099618,Male,Justin Solórzano,Mclean,VA,Virginia,22101,United States,North America,1992-02-16
15263,2099758,Male,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,1937-11-09
15264,2099862,Female,Lorenza Rush,Riverside,CA,California,92501,United States,North America,1937-10-12


In [11]:
df_customers.to_csv("cleaned_datasets/Customers.csv")

### 2] Exchange Rates Analysis

In [12]:
# load exchange rate table

df_exchange = pd.read_csv(list_of_csvs[1], encoding='ascii')
df_exchange

,Date,Currency,Exchange
0,1/1/2015,USD,1.0000
1,1/1/2015,CAD,1.1583
2,1/1/2015,AUD,1.2214
3,1/1/2015,EUR,0.8237
4,1/1/2015,GBP,0.6415
...,...,...,...
11210,2/20/2021,USD,1.0000
11211,2/20/2021,CAD,1.2610
11212,2/20/2021,AUD,1.2723
11213,2/20/2021,EUR,0.8238


In [13]:
df_exchange.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11215 entries, 0 to 11214
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      11215 non-null  object 
 1   Currency  11215 non-null  object 
 2   Exchange  11215 non-null  float64
dtypes: float64(1), object(2)
memory usage: 263.0+ KB


In [14]:
# Converting Date to date format

df_exchange['Date'] = pd.to_datetime(df_exchange['Date'], format='%m/%d/%Y')

In [15]:
# Checking null values

df_exchange.isna().sum()

Date        0
Currency    0
Exchange    0
dtype: int64

In [16]:
# Checking for duplicates

df_exchange.duplicated().sum()

0

In [17]:
df_exchange

,Date,Currency,Exchange
0,2015-01-01,USD,1.0000
1,2015-01-01,CAD,1.1583
2,2015-01-01,AUD,1.2214
3,2015-01-01,EUR,0.8237
4,2015-01-01,GBP,0.6415
...,...,...,...
11210,2021-02-20,USD,1.0000
11211,2021-02-20,CAD,1.2610
11212,2021-02-20,AUD,1.2723
11213,2021-02-20,EUR,0.8238


In [18]:
df_exchange.to_csv("cleaned_datasets/Exchange_Rates.csv")

### 3] Products Analysis

In [19]:
# load products table

df_products = pd.read_csv(list_of_csvs[2], encoding='ascii')
df_products

,ProductKey,Product Name,Brand,Color,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,$6.62,$12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,$6.62,$12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,$7.40,$14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,$11.00,$21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,$11.00,$21.57,101,MP4&MP3,1,Audio
...,...,...,...,...,...,...,...,...,...,...
2512,2513,Contoso Bluetooth Active Headphones L15 Red,Contoso,Red,$43.07,$129.99,505,Cell phones Accessories,5,Cell phones
2513,2514,Contoso Bluetooth Active Headphones L15 White,Contoso,White,$43.07,$129.99,505,Cell phones Accessories,5,Cell phones
2514,2515,Contoso In-Line Coupler E180 White,Contoso,White,$1.71,$3.35,505,Cell phones Accessories,5,Cell phones
2515,2516,Contoso In-Line Coupler E180 Black,Contoso,Black,$1.71,$3.35,505,Cell phones Accessories,5,Cell phones


In [20]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ProductKey      2517 non-null   int64 
 1   Product Name    2517 non-null   object
 2   Brand           2517 non-null   object
 3   Color           2517 non-null   object
 4   Unit Cost USD   2517 non-null   object
 5   Unit Price USD  2517 non-null   object
 6   SubcategoryKey  2517 non-null   int64 
 7   Subcategory     2517 non-null   object
 8   CategoryKey     2517 non-null   int64 
 9   Category        2517 non-null   object
dtypes: int64(3), object(7)
memory usage: 196.8+ KB


In [21]:
# Checking for null values

df_products.isna().sum()

ProductKey        0
Product Name      0
Brand             0
Color             0
Unit Cost USD     0
Unit Price USD    0
SubcategoryKey    0
Subcategory       0
CategoryKey       0
Category          0
dtype: int64

In [22]:
# Checking for duplicate values

df_products.duplicated().sum()

0

In [23]:
# Removing dollar signs and commas, then converting into float for both columns

df_products["Unit Cost USD"] = df_products["Unit Cost USD"].str.replace("[$,]", "", regex=True).astype(float)
df_products["Unit Price USD"] = df_products["Unit Price USD"].str.replace("[$,]", "", regex=True).astype(float)

In [24]:
# Replacing spaces in column name

df_products.columns = df_products.columns.str.replace(' ', '')

In [25]:
df_products

,ProductKey,ProductName,Brand,Color,UnitCostUSD,UnitPriceUSD,SubcategoryKey,Subcategory,CategoryKey,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,6.62,12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,6.62,12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,7.40,14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,11.00,21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,11.00,21.57,101,MP4&MP3,1,Audio
...,...,...,...,...,...,...,...,...,...,...
2512,2513,Contoso Bluetooth Active Headphones L15 Red,Contoso,Red,43.07,129.99,505,Cell phones Accessories,5,Cell phones
2513,2514,Contoso Bluetooth Active Headphones L15 White,Contoso,White,43.07,129.99,505,Cell phones Accessories,5,Cell phones
2514,2515,Contoso In-Line Coupler E180 White,Contoso,White,1.71,3.35,505,Cell phones Accessories,5,Cell phones
2515,2516,Contoso In-Line Coupler E180 Black,Contoso,Black,1.71,3.35,505,Cell phones Accessories,5,Cell phones


In [26]:
df_products.to_csv("cleaned_datasets/Products.csv")

### 4] Sales Analysis

In [27]:
# load sales table

df_sales = pd.read_csv(list_of_csvs[3], encoding='ascii')
df_sales

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,1/1/2016,NaN,265598,10,1304,1,CAD
1,366001,1,1/1/2016,1/13/2016,1269051,0,1048,2,USD
2,366001,2,1/1/2016,1/13/2016,1269051,0,2007,1,USD
3,366002,1,1/1/2016,1/12/2016,266019,0,1106,7,CAD
4,366002,2,1/1/2016,1/12/2016,266019,0,373,1,CAD
...,...,...,...,...,...,...,...,...,...
62879,2243030,1,2/20/2021,NaN,1216913,43,632,3,USD
62880,2243031,1,2/20/2021,2/24/2021,511229,0,98,4,EUR
62881,2243032,1,2/20/2021,2/23/2021,331277,0,1613,2,CAD
62882,2243032,2,2/20/2021,2/23/2021,331277,0,1717,2,CAD


In [28]:
# Replacing spaces in column name

df_sales.columns = df_sales.columns.str.replace(' ', '')

In [29]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   OrderNumber   62884 non-null  int64 
 1   LineItem      62884 non-null  int64 
 2   OrderDate     62884 non-null  object
 3   DeliveryDate  13165 non-null  object
 4   CustomerKey   62884 non-null  int64 
 5   StoreKey      62884 non-null  int64 
 6   ProductKey    62884 non-null  int64 
 7   Quantity      62884 non-null  int64 
 8   CurrencyCode  62884 non-null  object
dtypes: int64(6), object(3)
memory usage: 4.3+ MB


In [30]:
# Checking for null values

df_sales.isna().sum()

OrderNumber         0
LineItem            0
OrderDate           0
DeliveryDate    49719
CustomerKey         0
StoreKey            0
ProductKey          0
Quantity            0
CurrencyCode        0
dtype: int64

In [31]:
# Change Delivery Date and Order Date to Datetime format for analysis purpose

df_sales['DeliveryDate'] = pd.to_datetime(df_sales['DeliveryDate'], errors='coerce')
df_sales['OrderDate'] = pd.to_datetime(df_sales['OrderDate'])

In [32]:
# filling null delivery date values to same day as order date as it was bought in store, doing this only for non-online orders or can ask the company team in real world case

df_sales.loc[df_sales['DeliveryDate'].isna() & (df_sales['StoreKey'] != 0), 'DeliveryDate'] = df_sales['OrderDate']

In [33]:
df_sales

,OrderNumber,LineItem,OrderDate,DeliveryDate,CustomerKey,StoreKey,ProductKey,Quantity,CurrencyCode
0,366000,1,2016-01-01,2016-01-01,265598,10,1304,1,CAD
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD
...,...,...,...,...,...,...,...,...,...
62879,2243030,1,2021-02-20,2021-02-20,1216913,43,632,3,USD
62880,2243031,1,2021-02-20,2021-02-24,511229,0,98,4,EUR
62881,2243032,1,2021-02-20,2021-02-23,331277,0,1613,2,CAD
62882,2243032,2,2021-02-20,2021-02-23,331277,0,1717,2,CAD


In [34]:
df_sales.to_csv("cleaned_datasets/Sales.csv")

### 5] Stores Analysis

In [35]:
# load Stores table

df_stores = pd.read_csv(list_of_csvs[4], encoding='utf-8')
df_stores

,StoreKey,Country,State,Square Meters,Open Date
0,1,Australia,Australian Capital Territory,595.0,1/1/2008
1,2,Australia,Northern Territory,665.0,1/12/2008
2,3,Australia,South Australia,2000.0,1/7/2012
3,4,Australia,Tasmania,2000.0,1/1/2010
4,5,Australia,Victoria,2000.0,12/9/2015
...,...,...,...,...,...
62,63,United States,Utah,2000.0,3/6/2008
63,64,United States,Washington DC,1330.0,1/1/2010
64,65,United States,West Virginia,1785.0,1/1/2012
65,66,United States,Wyoming,840.0,1/1/2014


In [36]:
df_stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   StoreKey       67 non-null     int64  
 1   Country        67 non-null     object 
 2   State          67 non-null     object 
 3   Square Meters  66 non-null     float64
 4   Open Date      67 non-null     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 2.7+ KB


In [37]:
# Checking null values

df_stores.isna().sum()

StoreKey         0
Country          0
State            0
Square Meters    1
Open Date        0
dtype: int64

In [38]:
# Null values are for online as there is no Square Meters(traveling distance) for it

#  repalcing with zero
df_stores.fillna(0, inplace=True)

In [39]:
# Replacing spaces in column name

df_stores.columns = df_stores.columns.str.replace(' ', '')

In [40]:
df_stores['OpenDate'] = pd.to_datetime(df_stores['OpenDate'])

In [41]:
df_stores

,StoreKey,Country,State,SquareMeters,OpenDate
0,1,Australia,Australian Capital Territory,595.0,2008-01-01
1,2,Australia,Northern Territory,665.0,2008-01-12
2,3,Australia,South Australia,2000.0,2012-01-07
3,4,Australia,Tasmania,2000.0,2010-01-01
4,5,Australia,Victoria,2000.0,2015-12-09
...,...,...,...,...,...
62,63,United States,Utah,2000.0,2008-03-06
63,64,United States,Washington DC,1330.0,2010-01-01
64,65,United States,West Virginia,1785.0,2012-01-01
65,66,United States,Wyoming,840.0,2014-01-01


In [42]:
df_stores.to_csv("cleaned_datasets/Stores.csv")

## EDA

### 1] Customer_Analysis

In [43]:
from datetime import datetime

def calculate_age(birthday):
    today = datetime.now()
    age = today.year - birthday.year - ((today.month, today.day) < (birthday.month, birthday.day))
    return age

# Creating true copy of df_customers
df_customers_copy = df_customers.copy()
df_customers_copy['Age'] = df_customers_copy['Birthday'].apply(calculate_age)

# Define age bins and labels
bins = [0, 10, 20, 30, 40, 50, 60, float('inf')]
labels = ['Under 10', '10-19', '20-29', '30-39', '40-49', '50-59', '60 and above']

df_customers_copy['AgeRange'] = pd.cut(df_customers_copy['Age'], bins=bins, labels=labels, right=False)

In [44]:
df_customers_copy.head(3)

,CustomerKey,Gender,Name,City,StateCode,State,ZipCode,Country,Continent,Birthday,Age,AgeRange
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,1939-07-03,85,60 and above
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,1979-09-27,44,40-49
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,1947-05-26,77,60 and above


In [45]:
df_customers_copy.to_csv("dataset_upgrade/Customer_Age.csv")

### 2] Store Analysis

In [46]:
from datetime import datetime

# Calculate the number of days between today and Store OpenDate
def calculate_store_age_days(open_date):
    today = datetime.now()
    delta = today - open_date
    return delta.days

# Creating a true copy of df_stores
df_stores_copy = df_stores.copy()

# Calculating StoreAgeDays
df_stores_copy['StoreAgeDays'] = df_stores_copy['OpenDate'].apply(calculate_store_age_days)

# Calculate StoreAge
df_stores_copy['StoreAge'] = df_stores_copy['OpenDate'].apply(calculate_age)

In [47]:
df_stores_copy.head(3)

,StoreKey,Country,State,SquareMeters,OpenDate,StoreAgeDays,StoreAge
0,1,Australia,Australian Capital Territory,595.0,2008-01-01,6095,16
1,2,Australia,Northern Territory,665.0,2008-01-12,6084,16
2,3,Australia,South Australia,2000.0,2012-01-07,4628,12


In [48]:
df_stores_copy.to_csv("dataset_upgrade/Store_Age.csv")

### 3] Sales Analysis

In [49]:
import pandas as pd

# Load data into DataFrames
sales_df = pd.read_csv('cleaned_datasets/Sales.csv')
customers_df = pd.read_csv('cleaned_datasets/Customers.csv')

# Merge DataFrames based on CustomerKey
merged_df = pd.merge(customers_df, sales_df, on='CustomerKey', how='left')

# Calculate OrderCount
order_count = merged_df.groupby(['CustomerKey', 'Name']).size().reset_index(name='OrderCount')

# Customer Type based on purchase
def classify_customer(order_count):
    if order_count > 10:
        return "Loyal Customer"
    elif 5 <= order_count <= 10:
        return "Periodic Buyer"
    elif 1 <= order_count <= 4:
        return "VIP Customer"
    else:
        return "No Orders"

# Apply the classification
order_count['CustomerOrderingType'] = order_count['OrderCount'].apply(classify_customer)

# Sort by OrderCount
customer_order = order_count.sort_values(by='OrderCount', ascending=False)

In [50]:
customer_order.to_csv("dataset_upgrade/Customer_Order_details.csv")

In [51]:
# Yearly sales performance - Reveunue & Profit

#  Calculating Reveunue Per Customer USD and Profit Per Customer USD

df_sales['RevenueUSD'] = df_sales['Quantity'] * df_products['UnitPriceUSD']

#  By subtracting the total cost of goods sold from the Revenue USD, to calculate Profit USD

df_sales['ProfitUSD'] = df_sales['RevenueUSD']-(df_sales['Quantity'] * df_products['UnitCostUSD'])

In [52]:
df_sales.head(2)

,OrderNumber,LineItem,OrderDate,DeliveryDate,CustomerKey,StoreKey,ProductKey,Quantity,CurrencyCode,RevenueUSD,ProfitUSD
0,366000,1,2016-01-01,2016-01-01,265598,10,1304,1,CAD,12.99,6.37
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD,25.98,12.74


In [53]:
df_sales.to_csv("dataset_upgrade/Yearly sales performance.csv")

In [54]:
def rename_columns(df, column_mapping):
    existing_columns = set(df.columns)
    rename_dict = {old: new for old, new in column_mapping.items() if old in existing_columns}
    if rename_dict:
        df.rename(columns=rename_dict, inplace=True)

# Define column mappings for each DataFrame
customers_column_mapping = {
    'City': 'City_customer',
    'StateCode': 'StateCode_customer',
    'State': 'State_customer',
    'ZipCode': 'ZipCode_customer',
    'Country': 'Country_customer',
    'Continent': 'Continent_customer'
}

stores_column_mapping = {
    'Country': 'Country_store',
    'State': 'State_store'
}

# Apply the renaming function to the DataFrames
rename_columns(df_customers_copy, customers_column_mapping)
rename_columns(df_stores_copy, stores_column_mapping)

### Merge Datasets

In [55]:
df_SalesandStoreMerged = pd.merge(df_sales, df_stores_copy, on='StoreKey', how='left')

In [56]:
df_SalesStoresProductsMerged = pd.merge(df_SalesandStoreMerged, df_products, on='ProductKey', how='left')

In [57]:
df_SalesStoresProductsCustomersMerged = pd.merge(df_SalesStoresProductsMerged, df_customers_copy, on='CustomerKey', how='left')

In [58]:
df_finalMerged = pd.merge(df_SalesStoresProductsCustomersMerged, df_exchange,
                     left_on = ['OrderDate','CurrencyCode'],
                     right_on = ['Date','Currency'],
                     how = 'left')
df_finalMerged.head()

,OrderNumber,LineItem,OrderDate,DeliveryDate,CustomerKey,StoreKey,ProductKey,Quantity,CurrencyCode,RevenueUSD,...,State_customer,ZipCode_customer,Country_customer,Continent_customer,Birthday,Age,AgeRange,Date,Currency,Exchange
0,366000,1,2016-01-01,2016-01-01,265598,10,1304,1,CAD,12.99,...,Ontario,N5W 5K6,Canada,North America,1971-03-23,53,50-59,2016-01-01,CAD,1.3884
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD,25.98,...,California,93602,United States,North America,1995-11-20,28,20-29,2016-01-01,USD,1.0000
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD,14.52,...,California,93602,United States,North America,1995-11-20,28,20-29,2016-01-01,USD,1.0000
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD,150.99,...,Alberta,T4N 2A6,Canada,North America,1962-08-24,62,60 and above,2016-01-01,CAD,1.3884
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD,21.57,...,Alberta,T4N 2A6,Canada,North America,1962-08-24,62,60 and above,2016-01-01,CAD,1.3884


In [59]:
merged_data = df_finalMerged.to_csv("dataset_upgrade/Merged_data.csv")

In [60]:
df_finalMerged.drop(columns=['Date', 'Currency'], inplace=True)

In [61]:
df_finalMerged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   OrderNumber         62884 non-null  int64         
 1   LineItem            62884 non-null  int64         
 2   OrderDate           62884 non-null  datetime64[ns]
 3   DeliveryDate        62884 non-null  datetime64[ns]
 4   CustomerKey         62884 non-null  int64         
 5   StoreKey            62884 non-null  int64         
 6   ProductKey          62884 non-null  int64         
 7   Quantity            62884 non-null  int64         
 8   CurrencyCode        62884 non-null  object        
 9   RevenueUSD          2517 non-null   float64       
 10  ProfitUSD           2517 non-null   float64       
 11  Country_store       62884 non-null  object        
 12  State_store         62884 non-null  object        
 13  SquareMeters        62884 non-null  float64   

In [62]:
# Yearly sales performance - Reveunue & Profit

#  Calculating Reveunue Per Customer USD and Profit Per Customer USD

df_finalMerged['RevenueUSD'] = df_finalMerged['Quantity'] * df_finalMerged['UnitPriceUSD']

#  By subtracting the total cost of goods sold from the Revenue USD, to calculate Profit USD

df_finalMerged['ProfitUSD'] = df_finalMerged['RevenueUSD']-(df_finalMerged['Quantity'] * df_finalMerged['UnitCostUSD'])

In [63]:
# Creating a Frequency Column to the data

# Count the number of orders per customer
order_counts = df_finalMerged.groupby('CustomerKey')['OrderNumber'].count()

# function to Categorize frequency
def categorize_frequency(count):
    if count >= 1 and count <= 4:
        return 'Occasional'
    elif count >= 5 and count <= 10:
        return 'Moderate'
    elif count > 10:
        return 'Frequent'
    else:
        return 'Unknown'

# Apply categorization to get a Series of categories
order_counts_category = order_counts.apply(categorize_frequency)

# Reset index and Merge this frequency information back into the original DataFrame
order_counts_category = order_counts_category.reset_index(name='Frequency')

# Merge with the original DataFrame
df_finalMerged = df_finalMerged.merge(order_counts_category, on='CustomerKey', how='left')

In [64]:
# maximize display

pd.set_option('display.max_columns', None)

df_finalMerged.head(1)

,OrderNumber,LineItem,OrderDate,DeliveryDate,CustomerKey,StoreKey,ProductKey,Quantity,CurrencyCode,RevenueUSD,ProfitUSD,Country_store,State_store,SquareMeters,OpenDate,StoreAgeDays,StoreAge,ProductName,Brand,Color,UnitCostUSD,UnitPriceUSD,SubcategoryKey,Subcategory,CategoryKey,Category,Gender,Name,City_customer,StateCode_customer,State_customer,ZipCode_customer,Country_customer,Continent_customer,Birthday,Age,AgeRange,Exchange,Frequency
0,366000,1,2016-01-01,2016-01-01,265598,10,1304,1,CAD,68.0,36.73,Canada,Nunavut,1210.0,2015-04-04,3445,9,Contoso Lens Adapter M450 White,Contoso,White,31.27,68.0,406,Cameras & Camcorders Accessories,4,Cameras and camcorders,Male,Tyler Vaught,London,ON,Ontario,N5W 5K6,Canada,North America,1971-03-23,53,50-59,1.3884,Moderate


In [65]:
df_finalMerged.columns

Index(['OrderNumber', 'LineItem', 'OrderDate', 'DeliveryDate', 'CustomerKey',
       'StoreKey', 'ProductKey', 'Quantity', 'CurrencyCode', 'RevenueUSD',
       'ProfitUSD', 'Country_store', 'State_store', 'SquareMeters', 'OpenDate',
       'StoreAgeDays', 'StoreAge', 'ProductName', 'Brand', 'Color',
       'UnitCostUSD', 'UnitPriceUSD', 'SubcategoryKey', 'Subcategory',
       'CategoryKey', 'Category', 'Gender', 'Name', 'City_customer',
       'StateCode_customer', 'State_customer', 'ZipCode_customer',
       'Country_customer', 'Continent_customer', 'Birthday', 'Age', 'AgeRange',
       'Exchange', 'Frequency'],
      dtype='object')

In [66]:
df_finalMerged.head(3)

,OrderNumber,LineItem,OrderDate,DeliveryDate,CustomerKey,StoreKey,ProductKey,Quantity,CurrencyCode,RevenueUSD,ProfitUSD,Country_store,State_store,SquareMeters,OpenDate,StoreAgeDays,StoreAge,ProductName,Brand,Color,UnitCostUSD,UnitPriceUSD,SubcategoryKey,Subcategory,CategoryKey,Category,Gender,Name,City_customer,StateCode_customer,State_customer,ZipCode_customer,Country_customer,Continent_customer,Birthday,Age,AgeRange,Exchange,Frequency
0,366000,1,2016-01-01,2016-01-01,265598,10,1304,1,CAD,68.00,36.73,Canada,Nunavut,1210.0,2015-04-04,3445,9,Contoso Lens Adapter M450 White,Contoso,White,31.27,68.00,406,Cameras & Camcorders Accessories,4,Cameras and camcorders,Male,Tyler Vaught,London,ON,Ontario,N5W 5K6,Canada,North America,1971-03-23,53,50-59,1.3884,Moderate
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD,854.00,571.06,Online,Online,0.0,2010-01-01,5364,14,A. Datum SLR Camera X136 Silver,A. Datum,Silver,141.47,427.00,402,Digital SLR Cameras,4,Cameras and camcorders,Male,Frank Upchurch,Auberry,CA,California,93602,United States,North America,1995-11-20,28,20-29,1.0000,Occasional
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD,665.94,445.30,Online,Online,0.0,2010-01-01,5364,14,Fabrikam Microwave 1.5CuFt X1100 Black,Fabrikam,Black,220.64,665.94,803,Microwaves,8,Home Appliances,Male,Frank Upchurch,Auberry,CA,California,93602,United States,North America,1995-11-20,28,20-29,1.0000,Occasional


In [67]:
df_finalMerged.to_csv("cleaned_datasets/FinalMerged_data.csv")